In [1]:
import pydoc
import ast
import importlib
import re
import json

In [2]:
# Response from LLM
res = '''# This is a sample np.array(np.ones(5)) and another np.sum example
# Comment to check inline many functions

import numpy as np
from matplotlib.pyplot import show
 
x = np.linspace(0, 1, 100)
y = np.sin(2 * np.pi * x)

show()
pyplot.plot(x, y)
pyplot.show()
'''

# Extract imports from the file

In [3]:
packages = []
import_tree = ast.parse(res)

for node in ast.walk(import_tree):
    if 'names' in node.__dir__():
        # import numpy.array as npArray
        if 'module' in node.__dir__():
            # from matplotlib.pyplot import show
            packages.append({"module": node.module, "name": node.names[0].name})
        else:
            packages.append({"name": node.names[0].name, "asname": node.names[0].asname})

print(packages)

[{'name': 'numpy', 'asname': 'np'}, {'module': 'matplotlib.pyplot', 'name': 'show'}]


# Replace alias with package name

In [4]:
lines = res.splitlines()

In [5]:
nps = re.findall(fr'{packages[0]["asname"]}\.[a-zA-Z]+', res)

In [6]:
print(nps)

['np.array', 'np.ones', 'np.sum', 'np.linspace', 'np.sin', 'np.pi']


In [7]:
pakNames = []

In [8]:
'module' in packages[-1]

True

In [9]:
documentations = []

In [17]:
import sys
"matplotlib" in sys.modules

True

In [10]:
for package in packages:
    if "module" in package:
        package_module = package["module"]
        module_function = package["name"]
        
        spec = importlib.util.find_spec(package_module)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        function_definition = getattr(module, module_function)
        documentation = pydoc.render_doc(function_definition)
        documentations.append({"module": ".".join([package_module, module_function]), "documentation": documentation})
    else:
        last_module = None
        package_module = package["name"]
        module_function = package["asname"]

        if module_function is None:
            package_references = re.findall(rf"{package['name']}\.[a-zA-Z]+", res)
            package_references = [package_reference.rsplit('.', 1) for package_reference in package_references]
            
            for package_module, module_function in package_references:
                spec = importlib.util.find_spec(package_module)
                module = importlib.util.module_from_spec(spec)
                spec.loader.exec_module(module)
                function_definition = getattr(module, module_function)
                documentation = pydoc.render_doc(function_definition)
                documentaions.append({"module": ".".join([package_module, module_function]), "documentation": documentation})
        else:
            package_references = re.findall(rf"{package['asname']}\.[a-zA-Z]+", res)
            package_references = [re.sub(module_function, package_module, package_reference) for package_reference in package_references]    
            for package_reference in package_references:
                package_module, package_function = package_reference.split(".")
                if last_module != package_module:
                    spec = importlib.util.find_spec(package_module)
                    module = importlib.util.module_from_spec(spec)
                    spec.loader.exec_module(module)
                    last_module = module
                function_definition = getattr(module, package_function)
                documentation = pydoc.render_doc(function_definition)
                documentations.append({"module": package_reference, "documentation": documentation})

with open("example-documentation.json", "w") as json_file:
    json.dump(documentations, json_file)

/tmp/ipykernel_7756/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_7756/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_7756/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_7756/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  spec.loader.exec_module(module)
/tmp/ipykernel_7756/3044109395.py:37: UserWarning: The NumPy module was reloaded (imported a second time). This can in some case

In [19]:
import tiktoken
import 

In [25]:
enc = tiktoken.get_encoding("p50k_base")
here = enc.encode("\n".join([documentation["documentation"] for documentation in documentations]))

In [26]:
len(here)

7984

In [12]:
documentations

[{'module': 'numpy.array',
  'documentation': "Python Library Documentation: built-in function array in module numpy\n\na\x08ar\x08rr\x08ra\x08ay\x08y(...)\n    array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,\n          like=None)\n    \n    Create an array.\n    \n    Parameters\n    ----------\n    object : array_like\n        An array, any object exposing the array interface, an object whose\n        ``__array__`` method returns an array, or any (nested) sequence.\n        If object is a scalar, a 0-dimensional array containing object is\n        returned.\n    dtype : data-type, optional\n        The desired data-type for the array. If not given, NumPy will try to use\n        a default ``dtype`` that can represent the values (by applying promotion\n        rules when necessary.)\n    copy : bool, optional\n        If true (default), then the object is copied.  Otherwise, a copy will\n        only be made if ``__array__`` returns a copy, if obj is a nested\

In [11]:
spec = importlib.util.

SyntaxError: invalid syntax (2074393657.py, line 1)

In [ ]:
re.findall(r'np\.[a-zA-Z]+', res)

In [ ]:
print(documentations)

In [ ]:
pakDef = [re.sub(packages[0]["asname"], packages[0]["name"], np) for np in nps]

In [ ]:
pakDef

In [ ]:
module = importlib.import_module("matplotlib.pyplot")

In [ ]:
module = importlib.import_module("matplotlib")

In [ ]:
help(importlib.util.find_spec)

In [ ]:
help(importlib.util.module_from_spec)

In [ ]:
spec = importlib.util.find_spec("numpy")
print(help(spec.loader.exec_module))

In [ ]:
spec = importlib.util.find_spec("matplotlib")
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [ ]:
import matplotlib

In [ ]:
matplotlib.pyplot.show

In [ ]:
func = eval("matplotlib"+".pyplot")

In [ ]:
getattr(module, "pyplot.show")

In [ ]:
getattr(getattr(module, "pyplot.show"), "")

In [ ]:
for function in ["pyplot", "show"]:
    func = getattr(

In [ ]:
import matplotlib

In [ ]:
matplotlib.pyplot.__doc__

In [ ]:
import importlib.util
import inspect

def get_function_docs(module_name, function_name):
    try:
        # Import the module dynamically
        spec = importlib.util.find_spec(module_name)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        functions = function_name.split(".")

        getattr(getattr(module, functions[0]), function[1])

        func = getattr(module, functions[0])
        for function in functions[1:]:
            func = getattr(func, function)
        print(func)
        # Get the function dynamically
        function = getattr(module, function_name)

        # Use inspect to get documentation
        docstring = inspect.getdoc(func)
        return docstring
    except (ImportError, AttributeError):
        return None

if __name__ == "__main__":
    module_name = "matplotlib"
    function_name = "pyplot.show"

    docstring = get_function_docs(module_name, function_name)

    if docstring:
        print(f"Documentation for {module_name}.{function_name}:\n{docstring}\n")
    else:
        print(f"No documentation found for {module_name}.{function_name}\n")

In [ ]:
import matplotlib

In [ ]:
matplotlib.pyplot.show.__doc__

In [ ]:
help(getattr(getattr(module, "pyplot"), "show"))

In [ ]:
print(module.__doc__)

In [ ]:
module = importlib.import_module(packages[0]["name"])

In [ ]:
del numpy

In [ ]:
print(help(getattr(module, pakDef[0].split(".")[-1])))

In [ ]:
re.sub(fr'{packages[0]["asname"]}', packages[0]["name"], nps)

In [ ]:
print(pydoc.render_doc(re.search))

In [ ]:
print(pydoc.render_doc(re.match))

In [ ]:
re.sub(fr'{packages[0]["asname"]}\.[a-zA-Z]+', packages[0]['name'], res)

In [ ]:
print(help(re.sub))

In [ ]:
print(re.__doc__)

In [ ]:
print(help(re))

In [ ]:
print(pydoc.render_doc(re.sub))

In [ ]:
print(pydoc.render_doc(re.search))

# The problem is that there might be other definitionas similar to this that can not be detected

In [ ]:
print(re.Pattern.__doc__)
# BTW, This has huge doc for other two methods

In [ ]:
print(re.sub.__doc__)

In [ ]:
print(help(re.sub))

In [ ]:
print(pydoc.render_doc(re.sub))